# **Tuwaiq: Data Science and Machine Learning**

**Week #2 - Day #13**

Homework: Web Scraping Part 1

(https://scrapeme.live/shop/)

In [11]:
# ==========================
# Imports
# ==========================
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib3
import os
from urllib.parse import urlparse

In [12]:
# ==========================
# Step 1: Get shop main page
# ==========================
shop_url = "https://scrapeme.live/shop/"
shop_response = requests.get(shop_url)
shop_soup = BeautifulSoup(shop_response.text, "html.parser")

In [13]:
# ==========================
# Step 2: Collect all pages URLs
# ==========================
base_url = "https://scrapeme.live/shop"
pages_urls = [base_url] + [f"{base_url}/page/{i}/" for i in range(2, 49)]

In [14]:
# ==========================
# Step 3: Collect all product URLs
# ==========================
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
all_products_urls = []

for url in pages_urls:
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    products_soup = soup.find("ul", class_="products columns-4")
    products_urls = products_soup.find_all("a")[::2]  # extract product links
    all_products_urls.extend([a["href"] for a in products_urls])

In [15]:
# ==========================
# Step 4: Extract product details
# ==========================
product_images, name, price, description = [], [], [], []
status, in_stock, quantity, sku = [], [], [], []
categories, tags = [], []

for url in all_products_urls:
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    main_product = soup.find("main", class_="site-main")

    # Image
    product_img = main_product.find("img")
    product_images.append(product_img["src"].strip())

    # Summary section
    summary = main_product.find("div", class_="summary entry-summary")

    # Name
    name.append(summary.find("h1", class_="product_title entry-title").text.strip())

    # Price
    price.append(summary.find("span", class_="woocommerce-Price-amount amount").text.strip())

    # Description
    description.append(summary.find("div", class_="woocommerce-product-details__short-description").text.strip())

    # Status and stock
    status_text = summary.find("p", class_="stock in-stock").text.strip()
    status.append(status_text)
    in_stock.append("in stock" in status_text.lower())
    qty = status_text.lower().replace("in stock", "").strip()
    quantity.append(int(qty) if qty.isdigit() else 0)

    # Metadata
    meta = summary.find("div", class_="product_meta")
    sku.append(meta.find("span", class_="sku").text.strip())
    categories.append(meta.find("span", class_="posted_in").text.strip())
    tags.append(meta.find("span", class_="tagged_as").text.strip())

In [16]:
# ==========================
# Step 5: Download product images
# ==========================
os.makedirs("product_images", exist_ok=True)
image_paths = []

for img_url in product_images:
    if not img_url:
        image_paths.append(None)
        continue

    filename = os.path.basename(urlparse(img_url).path)
    path = os.path.join("product_images", filename)

    response = requests.get(img_url, stream=True, verify=False)
    if response.status_code == 200:
        with open(path, "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        print("Saved:", path)
        image_paths.append(path)
    else:
        print("Failed:", img_url)
        image_paths.append(None)

Saved: product_images/001.png
Saved: product_images/002.png
Saved: product_images/003.png
Saved: product_images/004.png
Saved: product_images/005.png
Saved: product_images/006.png
Saved: product_images/007.png
Saved: product_images/008.png
Saved: product_images/009.png
Saved: product_images/010.png
Saved: product_images/011.png
Saved: product_images/012.png
Saved: product_images/013.png
Saved: product_images/014.png
Saved: product_images/015.png
Saved: product_images/016.png
Saved: product_images/017.png
Saved: product_images/018.png
Saved: product_images/019.png
Saved: product_images/020.png
Saved: product_images/021.png
Saved: product_images/022.png
Saved: product_images/023.png
Saved: product_images/024.png
Saved: product_images/025.png
Saved: product_images/026.png
Saved: product_images/027.png
Saved: product_images/028.png
Saved: product_images/030.png
Saved: product_images/031.png
Saved: product_images/033.png
Saved: product_images/034.png
Saved: product_images/035.png
Saved: pro

In [17]:
# ==========================
# Step 6: Create dataframe
# ==========================
df = pd.DataFrame({
    "Product": name,
    "Description": description,
    "Price": price,
    "Availability": in_stock,
    "Quantity": quantity,
    "SKU": sku,
    "Category": categories,
    "Tag": tags
})

print(df.head())

      Product                                        Description    Price  \
0   Bulbasaur  Bulbasaur can be seen napping in bright sunlig...   £63.00   
1     Ivysaur  There is a bud on this Pokémon’s back. To supp...   £87.00   
2    Venusaur  There is a large flower on Venusaur’s back. Th...  £105.00   
3  Charmander  The flame that burns at the tip of its tail is...   £48.00   
4  Charmeleon  Charmeleon mercilessly destroys its foes using...  £165.00   

   Availability  Quantity   SKU                     Category  \
0          True        45  4391    Categories: Pokemon, Seed   
1          True       142  7227    Categories: Pokemon, Seed   
2          True        30  7036    Categories: Pokemon, Seed   
3          True       206  9086  Categories: Lizard, Pokemon   
4          True       284  6565   Categories: Flame, Pokemon   

                               Tag  
0  Tags: bulbasaur, Overgrow, Seed  
1    Tags: ivysaur, Overgrow, Seed  
2   Tags: Overgrow, Seed, venusaur  
3  T

In [18]:
# Save CSV
df.to_csv("Homework.csv", index=False)